## Defining models and their field types with Pydantic

### Standard field types

In [1]:
import pdb
import time

from pydantic import BaseModel


class Person(BaseModel):
    first_name: str
    last_name: str
    age: int

In [2]:
from datetime import date
from enum import Enum
from typing import List

from pydantic import BaseModel, ValidationError


class Gender(str, Enum):
    MALE = "MALE"
    FEMALE = "FEMALE"
    NON_BINARY = "NON_BINARY"


class Person(BaseModel):
    first_name: str
    last_name: str
    age: int
    gender: Gender
    birthday: date
    interests: List[str]

In [3]:
# invalid gender

try:
    p = Person(first_name="John", last_name='Doe', age=14, gender='INVALID_VALUE', birthday='1991-01-01',
               interests=['sport', 'reading'])
except ValidationError as e:
    print(str(e))

1 validation error for Person
gender
  value is not a valid enumeration member; permitted: 'MALE', 'FEMALE', 'NON_BINARY' (type=type_error.enum; enum_values=[<Gender.MALE: 'MALE'>, <Gender.FEMALE: 'FEMALE'>, <Gender.NON_BINARY: 'NON_BINARY'>])


In [4]:
from datetime import date

my_date = date.today()
print(my_date.isoformat())

2021-12-20


In [5]:
try:
    p = Person(first_name="John", last_name='Doe', age=14, gender='MALE', birthday='1991-13-01',
               interests=['sport', 'reading'])
except ValidationError as e:
    print(str(e))

1 validation error for Person
birthday
  invalid date format (type=value_error.date)


In [6]:
try:
    p = Person(first_name="John", last_name='Doe', age=14, gender='MALE', birthday='1991-11-01',
               interests=['sport', 'reading'])
except ValidationError as e:
    print(str(e))

In [7]:
p

Person(first_name='John', last_name='Doe', age=14, gender=<Gender.MALE: 'MALE'>, birthday=datetime.date(1991, 11, 1), interests=['sport', 'reading'])

In [8]:
class Address(BaseModel):
    street_address: str
    postal_code: str
    city: str
    country: str


class Person(BaseModel):
    first_name: str
    last_name: str
    age: int
    gender: Gender
    birthday: date
    address: Address

In [9]:
try:
    p = Person(first_name="John", last_name='Doe', age=14, gender='MALE', birthday='1991-11-01',
               address={
                   "street_address": 'gabdullin',
                   'postal_code': '050040',
                   'city': 'Almaty',
                   'country': 'KZ'
               })
except ValidationError as e:
    print(str(e))

In [10]:
p

Person(first_name='John', last_name='Doe', age=14, gender=<Gender.MALE: 'MALE'>, birthday=datetime.date(1991, 11, 1), address=Address(street_address='gabdullin', postal_code='050040', city='Almaty', country='KZ'))

### Optional fields and default values

In [11]:
from typing import Optional
from pydantic import BaseModel


class UserProfile(BaseModel):
    nickname: str
    location: Optional[str] = None
    subscribed_newsletter: bool = True


user = UserProfile(nickname='Joe')

In [12]:
user

UserProfile(nickname='Joe', location=None, subscribed_newsletter=True)

In [13]:
from datetime import datetime


class Model(BaseModel):
    # Don;t do this, this example shows you why it doesn't work
    d: datetime = datetime.now()


o1 = Model()

In [14]:
o1

Model(d=datetime.datetime(2021, 12, 20, 17, 30, 22, 42881))

In [15]:
import time

time.sleep(1)

In [16]:
o2 = Model()

In [17]:
o2

Model(d=datetime.datetime(2021, 12, 20, 17, 30, 22, 42881))

In [18]:
print(o1.d < o2.d)

False


Even though we waited for 1 second between the instantiation of o1 and o2, the d datetime is the same!

 Pydantic provides a Fieldfunction that allows us to set some advanced options on our fields, including one to set a factory for creating dynamic values.

### Field validation

In [19]:
from typing import Optional

from pydantic import BaseModel, Field, ValidationError


class Person(BaseModel):
    first_name: str = Field(..., min_length=3)
    last_name: str = Field(..., min_length=3)
    age: Optional[int] = Field(None, ge=0, le=150)

The first positional argument defines the default value for the field. If the field is required, we use the ellipsis .... Then, the keyword arguments are there to set options for the field, including some basic validation

### Dynamic default values

In [20]:
from datetime import datetime
from typing import List

from pydantic import BaseModel, Field


def list_factory():
    return ["a", "b", "c"]


class Model(BaseModel):
    l: List[str] = Field(default_factory=list_factory)
    d: datetime = Field(default_factory=datetime.now)
    l2: List[str] = Field(default_factory=list)

This argument expects you to pass a function that will be called during model instantiation. Thus, the resulting object will be evaluated at runtime each time you create a new object.

In [21]:
m = Model()

m

In [22]:
m

Model(l=['a', 'b', 'c'], d=datetime.datetime(2021, 12, 20, 17, 30, 31, 353978), l2=[])

In [23]:
m1 = Model()

In [24]:
m1

Model(l=['a', 'b', 'c'], d=datetime.datetime(2021, 12, 20, 17, 30, 32, 12601), l2=[])

## Validating email address and URLs with Pydantic types

In [25]:
!pip install email-validator

     |████████████████████████████████| 241 kB 707 kB/s            


In [26]:
from pydantic import BaseModel, EmailStr, HttpUrl, ValidationError


class User(BaseModel):
    email: EmailStr
    website: HttpUrl


In [27]:
try:
    User(email="jon", website='https://azat.ai')
except ValidationError as e:
    print(e)

1 validation error for User
email
  value is not a valid email address (type=value_error.email)


In [28]:
try:
    User(email="jon@gmail.", website='https://azat.ai')
except ValidationError as e:
    print(e)

1 validation error for User
email
  value is not a valid email address (type=value_error.email)


In [29]:
try:
    User(email="jon@azat.ai", website='https://azat.ai')
except ValidationError as e:
    print(e)

In [30]:
try:
    User(email="jon@azat.ai", website='https://azat')
except ValidationError as e:
    print(e)

1 validation error for User
website
  URL host invalid, top level domain required (type=value_error.url.host)


## Creating Model Variations with Class Inheritance

In [31]:
from pydantic import BaseModel


class PostCreate(BaseModel):
    title: str
    content: str


class PostPublic(BaseModel):
    id: int
    title: str
    content: str


class PostDB(BaseModel):
    id: int
    title: str
    content: str
    nb_views: int = 0

PostCreate will be used for a POST endpoint to create a new post. We expect the user to give the title and the content; however, the identifier(ID) will be automatically determined by the database

PostPublic will be used when we retrieve the data of a post. We want its title and content, of course, but also its associated ID in the database

PostDB will carry all the data we wish to store in the database. Here, we also want to store the number of views, but we want to keep this secret to make our own statistics internally

In [32]:
from pydantic import BaseModel


class PostBase(BaseModel):
    title: str
    content: str


class PostCreate(PostBase):
    pass


class PostDB(PostBase):
    id: int
    views: int = 0


class PostPublic(PostBase):
    id: int

It's also very convenient if you wish to define methods on your model. Remember that Pydantic models are regular Python classes, so you can implement as many methods as you wish!

In [33]:
class PostBase(BaseModel):
    title: str
    content: str

    def excerpt(self) -> str:
        return f"{self.content[:140]}"

## Adding Custom Data Validation With Pydantic

### Applying validation at a field level


In [4]:
from datetime import date

from pydantic import BaseModel, validator


class Person(BaseModel):
    first_name: str
    last_name: str
    birthdate: date

    @validator("birthdate")
    def valid_birthdate(cls, v: date):
        delta = date.today() - v
        age = delta.days / 365
        if age > 120:
            raise ValueError("You must a bit too old!")
        return v



### Applying validation at an Object Level

It happens quite often that the validation of one field is dependent on another—for example, to check if a password confirmation matches the password or to enforce a field to be required in certain circumstances.

In [35]:
# we need to access the whole object data, not only one fied sometimes.

from pydantic import BaseModel, EmailStr, ValidationError, root_validator


class UserRegistration(BaseModel):
    email: EmailStr
    password: str
    password_confirmation: str

    @root_validator()
    def passwords_match(cls, values):
        password = values.get("password")
        password_confirmation = values.get("password_confirmation")
        if password != password_confirmation:
            raise ValueError("Passwords don't match")
        return values

### Applying validation before pydantic parsing

In [1]:
from typing import List
from pydantic import BaseModel, validator


class Model(BaseModel):
    values: List[int]

    @validator("values", pre=True)
    def split_string_values(cls, v):
        if isinstance(v, str):
            return v.split(",")
        return v

In [2]:
m = Model(values="1,2,3")
print(m.values)

[1, 2, 3]


## Working with Pydantic objects

### Converting an object into dictionary

In [6]:
person = Person(
    first_name="John",
    last_name="doe",
    birthdate="1991-01-01"
)

In [8]:
person

Person(first_name='John', last_name='doe', birthdate=datetime.date(1991, 1, 1))

In [9]:
person_dict = person.dict()

In [10]:
person_dict

{'first_name': 'John',
 'last_name': 'doe',
 'birthdate': datetime.date(1991, 1, 1)}

In [11]:
person.dict(include={"first_name"})

{'first_name': 'John'}

In [12]:
person.dict(exclude={"birthdate"})

{'first_name': 'John', 'last_name': 'doe'}

The include and exclude arguments expect a set with the keys of the fields you want to include or exclude

### Creating an instance from a sub-clas object

In [13]:
from pydantic import BaseModel


class PostBase(BaseModel):
    # base module for repeated content
    title: str
    content: str


class PostCreate(PostBase):
    #  for http post creation endpoint ( the data we need to get from the form)
    pass


class PostPublic(PostBase):
    # for http get endpoint
    id: int


class PostDB(PostBase):
    # for storing inside the database
    id: int
    nb_views: int = 0

In [32]:
# creating PostDB instance before storing it in the database
from fastapi import FastAPI, status

app = FastAPI()

# fake db
db_posts = {

}


@app.post("/posts/", status_code=status.HTTP_201_CREATED, response_model=PostPublic)
async def create_post(post_create: PostCreate):
    new_id = max(len(db_posts.keys()) or (0,)) + 1
    post = PostDB(id=new_id, **post_create.dict())

    db_posts[new_id] = post
    return post

In [33]:
import nest_asyncio

nest_asyncio.apply()

In [34]:
import uvicorn

In [35]:

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [37418]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:59990 - "POST /posts HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:59990 - "POST /posts/ HTTP/1.1" 201 Created
INFO:     127.0.0.1:60032 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:60032 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [37418]


![2E585r](https://raw.githubusercontent.com/azataiot/images/master/2021/12/20/2E585r.png)

we first determine the missing id property, which is given to us by the database.

The effect of ** in a function call is to transform a dictionary into keyword arguments.

### Updating an instance with a partial one (HTTP PATCH)

In [58]:
from typing import Optional


class PostBase(BaseModel):
    title: str
    content: str


class PostPublic(PostBase):
    pass


# dummy db
posts = {
    1: PostBase(title="Hello", nb_views=100, content='some tons'),
}


class PostPartialUpdate(BaseModel):
    title: Optional[str] = None
    content: Optional[str] = None


In [59]:
from fastapi import HTTPException

app = FastAPI()


@app.patch("/posts/{id}", response_model=PostPublic)
async def partial_update(post_id: int, post_update: PostPartialUpdate):
    try:
        _posts = posts[post_id]

        updated_fields = post_update.dict(exclude_unset=True)
        updated_post = _posts.copy(update=updated_fields)

        posts[post_id] = updated_post
        print(updated_post)
        print(posts)
        return updated_post
    except KeyError:
        raise HTTPException(status.HTTP_404_NOT_FOUND)

In [60]:
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [37418]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:60338 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:60338 - "GET /openapi.json HTTP/1.1" 200 OK
title='string' content='string'
{1: PostBase(title='string', content='string')}
INFO:     127.0.0.1:60338 - "PATCH /posts/%7Bid%7D?post_id=1 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [37418]
